In [2]:
library(rsample)
library(recipes)
library(h2o)

In [3]:
ames <- AmesHousing::make_ames()
set.seed(123) 
split <- initial_split(ames, strata = "Sale_Price")
ames_train <- training(split)
ames_test <- testing(split)

In [10]:
h2o.no_progress()
h2o.init()

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         15 hours 41 minutes 
    H2O cluster timezone:       Asia/Seoul 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.36.0.4 
    H2O cluster version age:    8 days  
    H2O cluster name:           H2O_started_from_R_apf_temp_admin_tgm898 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   8.59 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:                  R version 4.1.3 (2022-03-10) 



- step_other() - 범주형 변수의 level이 여러개일 때, 하위 범주를 기타로 묶음

    -threshold = 0.05 - 하위 5% 범주는 기타로 묶임


In [4]:
blueprint <- recipe(Sale_Price ~ ., data = ames_train) %>%
    step_other(all_nominal(), threshold = 0.005)

In [11]:
train_h2o <- prep(blueprint, training = ames_train, retain = T) %>%
    juice() %>%
    as.h2o()
test_h2o <- prep(blueprint, training = ames_test) %>%
    juice() %>%
    as.h2o()

In [12]:
Y <- "Sale_Price" ; X <- setdiff(names(ames_train), Y)

## 누적 모델
- 여러 기본 학습기의 예측을 결합

### 공통 앙상블 방법
- 하나의 최상의 모델을 선택하는 것이 아닌, 여러 모델을 결합

### 슈퍼 러닝 알고리즘

1. 앙상블 구성
    - 학습기로 구성된 목록 L 구성
    - 메타 학습 알고리즘 구성 (대부분은 일종의 정규화 회귀)
2. 훈련
    - 각각의 각습기에 대해 K-fold CV수행
    - 각각에서 교차 검증된 예측 수집
3. 예측
    - 앙상블 예측을 생성하려면 먼저 기본 학습기로부터 예측을 생성
    - 앙상블 예측을 생성하기 위해 이러한 예측을 메타 학습기에 제공

### 기존 모델 쌓기 (Stacking)

#### 1. 정규화 회귀
#### 2. 랜덤포레스트
#### 3. GBM
#### 4. XGBoost

- 각 모델을 개별적으로 훈련
1. 모든 모델은 동일한 훈련 세트에서 훈련
2. 모든 모델은 동일한 수의 CV폴드로 훈련
3. 모든 모델은 동일한 관측값이 사용되도록 동일한 fold 할당을 사용
- ```fold_assignment = "Modulo"```
4. 모든 모델의 교차검증된 예측값은 보존되어야 함
- ```keep_cross_validation_prerdictions = T```

In [15]:
best_glm <- h2o.glm(
    x = X, y = Y, 
    training_frame = train_h2o, alpha = 0.1,
    remove_collinear_columns = TRUE, nfolds = 10, 
    fold_assignment = "Modulo",
    keep_cross_validation_predictions = TRUE, 
    seed = 123
)

In [ ]:
best_rf <- h2o.randomForest(
    x = X, y = Y, training_frame = train_h2o, 
    ntrees = 1000, mtries = 20,
    max_depth = 30, min_rows = 1, sample_rate = 0.8, nfolds = 10,
    fold_assignment = "Modulo", keep_cross_validation_predictions = TRUE,
    seed = 123, 
    stopping_rounds = 50, stopping_metric = "RMSE", stopping_tolerance = 0
)

In [ ]:
best_gbm <- h2o.gbm(
    x = X, y = Y, training_frame = train_h2o, 
    ntrees = 5000, learn_rate = 0.01,
    max_depth = 7, min_rows = 5, sample_rate = 0.8, nfolds = 10,
    fold_assignment = "Modulo", keep_cross_validation_predictions = TRUE,
    seed = 123, 
    stopping_rounds = 50, stopping_metric = "RMSE", stopping_tolerance = 0
)

In [ ]:
best_xgb <- h2o.xgboost(
    x = X, y = Y, training_frame = train_h2o, 
    ntrees = 5000, learn_rate = 0.05,
    max_depth = 3, min_rows = 3, sample_rate = 0.8, categorical_encoding = "Enum",
    nfolds = 10, fold_assignment = "Modulo", 
    keep_cross_validation_predictions = TRUE, 
    seed = 123, 
    stopping_rounds = 50, stopping_metric = "RMSE", stopping_tolerance = 0
)

### 앙상블 모형 생성 & 적합
### ```h2o.stackedEnsemble()```
- 모델 스택
- ```base_models = list()``` 
- ```metalearner_algorithm``` : ```"AUTO"```, ```"deeplearning"```, ```"drf"```, ```"gbm"```, ```"glm"```, ```"naivebayes"```, ```"xgboost"```

- 메타 학습 알고리즘으로 랜덤포레스트 적용

In [14]:
ensemble_tree <- h2o.stackedEnsemble(
  x = X, y = Y, training_frame = train_h2o, model_id = "my_tree_ensemble",
  base_models = list(best_glm, best_rf, best_gbm, best_xgb),
  metalearner_algorithm = "drf"
)

In [ ]:
ensemble_tree
attributes(ensemble_tree)

### 앙상블 모형 평가

In [ ]:
h2o.performance(ensemble_tree, newdata = test_h2o)

### 학습기별 예측값의 상관관계

In [ ]:
glm_res <- h2o.performance(best_glm, newdata = test_h2o)
rf_res <- h2o.performance(best_rf, newdata = test_h2o)
gbm_res <- h2o.performance(best_gbm, newdata = test_h2o)
xgb_res <- h2o.performance(best_xgboost, newdata = test_h2o)

### 그리드 서치 Stacking
#### - 그리드 서치에서 각 모델의 이점을 활용하여 메타모델 생성 
#### - 각 그리드서치 모델의 결과에서 높은 변동성을 보이는 경우 유용

In [ ]:
hyper_grid <- list(
  max_depth = c(1, 3, 5),
  min_rows = c(1, 5, 10),
  learn_rate = c(0.01, 0.05, 0.1),
  learn_rate_annealing = c(0.99, 1),
  sample_rate = c(0.5, 0.75, 1),
  col_sample_rate = c(0.8, 0.9, 1)
)

In [ ]:
search_criteria <- list(
  strategy = "RandomDiscrete",
  max_models = 25
)

In [ ]:
random_grid <- h2o.grid(
  algorithm = "gbm", grid_id = "gbm_grid", x = X, y = Y,
  training_frame = train_h2o, hyper_params = hyper_grid,
  search_criteria = search_criteria, ntrees = 5000, stopping_metric = "RMSE",     
  stopping_rounds = 10, stopping_tolerance = 0, nfolds = 10, 
  fold_assignment = "Modulo", keep_cross_validation_predictions = TRUE,
  seed = 123
)

#### - hyper_grid에서 랜덤으로 조합된 25개의 조합으로 25개의 모형을 생성

In [ ]:
h2o.getGrid(
  grid_id = "gbm_grid", 
  sort_by = "rmse"
)

#### 25개중 최적 모델

In [ ]:
best_model_id <- random_grid_perf@model_ids[[1]]
best_model <- h2o.getModel(best_model_id)
h2o.performance(best_model, newdata = test_h2o)

#### 최적의모델 1개만 사용하는 대신, 그리드서치의 모든 모델을 결합
- ```base_models```를 그리드서치의 모델들로

In [ ]:
ensemble <- h2o.stackedEnsemble(
    x = X, y = Y, training_frame = train_h2o, model_id = "ensemble_gbm_grid",
    base_models = random_grid@model_ids, # 그리드서치의 모든 모델
    metalearner_algorithm = "gbm"
)

### 자동화된 머신러닝 (AutoML)
- 여러 학습기에 대해 자동화된 검색을 수행한 다음 결과 모델을 쌓는 것

### ```h2o.automl()```

In [2]:
auto_ml <- h2o.automl(
  x = X, y = Y, training_frame = train_h2o, nfolds = 5, 
  max_runtime_secs = 60 * 10, max_models = 50,
  keep_cross_validation_predictions = TRUE, sort_metric = "RMSE", 
    seed = 123,
  stopping_rounds = 50, stopping_metric = "RMSE", stopping_tolerance = 0
)

In [ ]:
auto_ml

In [ ]:
attributes(auto_ml)

In [ ]:
auto_ml@leaderboard %>% 
  as.data.frame() %>%
  dplyr::select(model_id, rmse) %>%
  dplyr::slice(1:25)